In [330]:
%load_ext autoreload
%autoreload
from simba import transfer_function_to_graph, tf2rss, adiabatically_eliminate
from sympy import symbols, simplify, Matrix, sqrt, conjugate, lambdify

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Tuned cavity

In [331]:
s = symbols('s')
gamma_f = symbols('gamma_f', real=True, positive=True)
tf = (s + gamma_f) / (s - gamma_f)
split_network = tf2rss(tf).to_slh().split()
gamma, = split_network.aux_coupling_constants

In [333]:
split_network.state_vector

Matrix([
[              a],
[   conjugate(a)],
[             a'],
[  conjugate(a')],
[            ain],
[ conjugate(ain)],
[           aout],
[conjugate(aout)]])

In [318]:
split_network.dynamical_matrix.eqns

Matrix([
[                                                                        -sqrt(gamma)*sqrt(gamma_f)*a'/s],
[                                                             -sqrt(gamma)*sqrt(gamma_f)*conjugate(a')/s],
[                                 sqrt(gamma)*sqrt(gamma_f)*a/s - sqrt(2)*sqrt(gamma)*ain/s + gamma*a'/s],
[sqrt(gamma)*sqrt(gamma_f)*conjugate(a)/s - sqrt(2)*sqrt(gamma)*conjugate(ain)/s + gamma*conjugate(a')/s],
[                                                                                                      0],
[                                                                                                      0],
[                                                                          -sqrt(2)*sqrt(gamma)*a' + ain],
[                                                    -sqrt(2)*sqrt(gamma)*conjugate(a') + conjugate(ain)]])

In [319]:
tf = split_network.tfm['aout', 'ain']
tf

(-gamma*gamma_f/s**2 - gamma/s - 1)/(-gamma*gamma_f/s**2 + gamma/s - 1)

In [320]:
adiabatically_eliminate(tf, gamma).simplify()

(gamma_f + s)/(gamma_f - s)

In [328]:
tf = split_network.tfm['aout', 'a'].simplify()
tf

-sqrt(2)*gamma*sqrt(gamma_f)*s/(gamma*gamma_f - gamma*s + s**2)

In [329]:
adiabatically_eliminate(tf, gamma).simplify()

sqrt(2)*sqrt(gamma_f)*s/(-gamma_f + s)

In [324]:
tf = split_network.tfm['a', 'ain'].simplify()
tf

sqrt(2)*gamma*sqrt(gamma_f)/(gamma*gamma_f - gamma*s + s**2)

In [325]:
adiabatically_eliminate(tf, gamma).simplify()

sqrt(2)*sqrt(gamma_f)/(gamma_f - s)

In [323]:
split_network.interaction_hamiltonian.h

Matrix([
[0, 0, I*sqrt(gamma)*sqrt(gamma_f),                            0],
[0, 0,                           0, -I*sqrt(gamma)*sqrt(gamma_f)],
[0, 0,                           0,                            0],
[0, 0,                           0,                            0]])

First looking at passive realisation of coupled cavity setup with coupling constant $g = 0$

In [279]:
s = symbols('s')
gamma_f, omega_s = symbols('gamma_f omega_s', real=True, positive=True)
tf = (s**2 + s * gamma_f + omega_s**2) / (s**2 - s * gamma_f + omega_s**2)

transfer_function_to_graph(tf, 'passive_coupled_cavity.png', layout='dot')

wrote passive_coupled_cavity.png


![](passive_coupled_cavity.png)

In [280]:
split_network = tf2rss(tf).to_slh().split()

In [281]:
h_int = split_network.interaction_hamiltonian
h_int.expr.simplify()

I*(-sqrt(gamma_1)*sqrt(gamma_f)*a_1*conjugate(a'_1) + sqrt(gamma_1)*sqrt(gamma_f)*conjugate(a_1)*a'_1 + omega_s*a_1*conjugate(a_2) - omega_s*conjugate(a_1)*a_2)

In [282]:
split_network.interaction_hamiltonian.h

Matrix([
[0, 0, -I*omega_s,         0, I*sqrt(gamma_1)*sqrt(gamma_f),                              0, 0, 0],
[0, 0,          0, I*omega_s,                             0, -I*sqrt(gamma_1)*sqrt(gamma_f), 0, 0],
[0, 0,          0,         0,                             0,                              0, 0, 0],
[0, 0,          0,         0,                             0,                              0, 0, 0],
[0, 0,          0,         0,                             0,                              0, 0, 0],
[0, 0,          0,         0,                             0,                              0, 0, 0],
[0, 0,          0,         0,                             0,                              0, 0, 0],
[0, 0,          0,         0,                             0,                              0, 0, 0]])

In [283]:
h_int.states

Matrix([
[            a_1],
[ conjugate(a_1)],
[            a_2],
[ conjugate(a_2)],
[           a'_1],
[conjugate(a'_1)],
[           a'_2],
[conjugate(a'_2)]])

In [284]:
simplify(h_int.dynamical_matrix)

Matrix([
[                           0,                            0, -omega_s,        0, sqrt(gamma_1)*sqrt(gamma_f),                           0, 0, 0],
[                           0,                            0,        0, -omega_s,                           0, sqrt(gamma_1)*sqrt(gamma_f), 0, 0],
[                     omega_s,                            0,        0,        0,                           0,                           0, 0, 0],
[                           0,                      omega_s,        0,        0,                           0,                           0, 0, 0],
[-sqrt(gamma_1)*sqrt(gamma_f),                            0,        0,        0,                           0,                           0, 0, 0],
[                           0, -sqrt(gamma_1)*sqrt(gamma_f),        0,        0,                           0,                           0, 0, 0],
[                           0,                            0,        0,        0,                           0,      

In [285]:
split_network.frequency_domain_eqns

Matrix([
[                                                                            sqrt(gamma_1)*sqrt(gamma_f)*a'_1 - omega_s*a_2 + s*a_1],
[                                           sqrt(gamma_1)*sqrt(gamma_f)*conjugate(a'_1) - omega_s*conjugate(a_2) + s*conjugate(a_1)],
[                                                                                                               omega_s*a_1 + s*a_2],
[                                                                                         omega_s*conjugate(a_1) + s*conjugate(a_2)],
[                                            -sqrt(gamma_1)*sqrt(gamma_f)*a_1 + sqrt(2)*sqrt(gamma_1)*ain_1 - gamma_1*a'_1 + s*a'_1],
[-sqrt(gamma_1)*sqrt(gamma_f)*conjugate(a_1) + sqrt(2)*sqrt(gamma_1)*conjugate(ain_1) - gamma_1*conjugate(a'_1) + s*conjugate(a'_1)],
[                                                                                                                            s*a'_2],
[                                                    

Looking at adiabatic elimination of $a_1'$

$\dot{a}_1' = -\gamma_1 a_1' - \sqrt{\gamma_1 \gamma_f} a_1 + \sqrt{2 \gamma_1} a_\text{in}$

adiabatic elimination: $\dot{a}_1' = 0$

$a_1' = \sqrt{\frac{\gamma_f}{\gamma_1}} a_1 - \sqrt{\frac{2}{\gamma_1}} a_\text{in}$

$H_\text{int} = i \sqrt{2\gamma_f}(a_\text{in}^\dagger a_1 - a_\text{in} a_1^\dagger)$

In [303]:
split_network.dynamical_matrix.states.states

Matrix([
[              a_1],
[   conjugate(a_1)],
[              a_2],
[   conjugate(a_2)],
[             a'_1],
[  conjugate(a'_1)],
[             a'_2],
[  conjugate(a'_2)],
[            ain_1],
[ conjugate(ain_1)],
[           aout_1],
[conjugate(aout_1)],
[            ain_2],
[ conjugate(ain_2)],
[           aout_2],
[conjugate(aout_2)]])

In [305]:
split_network.dynamical_matrix.eqns

Matrix([
[                                                                -sqrt(gamma_1)*sqrt(gamma_f)*a'_1/s + omega_s*a_2/s],
[                                          -sqrt(gamma_1)*sqrt(gamma_f)*conjugate(a'_1)/s + omega_s*conjugate(a_2)/s],
[                                                                                                     -omega_s*a_1/s],
[                                                                                          -omega_s*conjugate(a_1)/s],
[                                 sqrt(gamma_1)*sqrt(gamma_f)*a_1/s - sqrt(2)*sqrt(gamma_1)*ain_1/s + gamma_1*a'_1/s],
[sqrt(gamma_1)*sqrt(gamma_f)*conjugate(a_1)/s - sqrt(2)*sqrt(gamma_1)*conjugate(ain_1)/s + gamma_1*conjugate(a'_1)/s],
[                                                                                                                  0],
[                                                                                                                  0],
[                                      

In [307]:
# Calculating the input-output transfer function
tfm = split_network.tfm

tf = tfm['aout_1', 'ain_1'].simplify()
gamma_1, _ = split_network.aux_coupling_constants
adiabatically_eliminate(tf, gamma_1)

(gamma_f*s + omega_s**2 + s**2)/(gamma_f*s - omega_s**2 - s**2)

In [308]:
tf = tfm['aout_1', 'a_1'].simplify()
gamma_1, _ = split_network.aux_coupling_constants
adiabatically_eliminate(tf, gamma_1)

sqrt(2)*sqrt(gamma_f)*s**3/(-gamma_f*s**2 + omega_s**2*s + s**3)

Now looking at the active realisation ($g \neq 0$)

In [309]:
# parameterise with lambda = g**2 - omega_s**2 > 0
lmbda = symbols('lambda', real=True, positive=True)
tf = (s**2 + s * gamma_f - lmbda) / (s**2 - s * gamma_f - lmbda)

transfer_function_to_graph(tf, 'active_coupled_cavity.png', layout='dot')

wrote active_coupled_cavity.png


![](active_coupled_cavity.png)

In [310]:
split_network = tf2rss(tf).to_slh().split()
h_int = split_network.interaction_hamiltonian
h_int.expr.simplify()

I*(-sqrt(gamma_1)*sqrt(gamma_f)*a_1*conjugate(a'_1) + sqrt(gamma_1)*sqrt(gamma_f)*conjugate(a_1)*a'_1 - sqrt(lambda)*a_1*a_2 + sqrt(lambda)*conjugate(a_1)*conjugate(a_2))

In [311]:
simplify(h_int.dynamical_matrix)

Matrix([
[                           0,                            0,            0, sqrt(lambda), sqrt(gamma_1)*sqrt(gamma_f),                           0, 0, 0],
[                           0,                            0, sqrt(lambda),            0,                           0, sqrt(gamma_1)*sqrt(gamma_f), 0, 0],
[                           0,                 sqrt(lambda),            0,            0,                           0,                           0, 0, 0],
[                sqrt(lambda),                            0,            0,            0,                           0,                           0, 0, 0],
[-sqrt(gamma_1)*sqrt(gamma_f),                            0,            0,            0,                           0,                           0, 0, 0],
[                           0, -sqrt(gamma_1)*sqrt(gamma_f),            0,            0,                           0,                           0, 0, 0],
[                           0,                            0,       

In [313]:
split_network.frequency_domain_eqns

Matrix([
[                                                            sqrt(gamma_1)*sqrt(gamma_f)*a'_1 + sqrt(lambda)*conjugate(a_2) + s*a_1],
[                                                 sqrt(gamma_1)*sqrt(gamma_f)*conjugate(a'_1) + sqrt(lambda)*a_2 + s*conjugate(a_1)],
[                                                                                               sqrt(lambda)*conjugate(a_1) + s*a_2],
[                                                                                               sqrt(lambda)*a_1 + s*conjugate(a_2)],
[                                            -sqrt(gamma_1)*sqrt(gamma_f)*a_1 + sqrt(2)*sqrt(gamma_1)*ain_1 - gamma_1*a'_1 + s*a'_1],
[-sqrt(gamma_1)*sqrt(gamma_f)*conjugate(a_1) + sqrt(2)*sqrt(gamma_1)*conjugate(ain_1) - gamma_1*conjugate(a'_1) + s*conjugate(a'_1)],
[                                                                                                                            s*a'_2],
[                                                    

In [314]:
# Calculating the input-output transfer function
tfm = split_network.tfm

tf = tfm['aout_1', 'ain_1'].simplify()
gamma_1, _ = split_network.aux_coupling_constants
adiabatically_eliminate(tf, gamma_1)

(gamma_f*s - lambda + s**2)/(gamma_f*s + lambda - s**2)

In [19]:
(s**2 + s * gamma_f - lmbda) / (s**2 - s * gamma_f - lmbda)

(gamma_f*s - lambda + s**2)/(-gamma_f*s - lambda + s**2)

Differs by phase shift of $\pi$

Now let's look at the transfer function from $a_1$ to $aout_1$, expect it to be frequency independent

In [315]:
tf = tfm['aout_1', 'a_1'].simplify()
gamma_1, _ = split_network.aux_coupling_constants
adiabatically_eliminate(tf, gamma_1)

-sqrt(2)*sqrt(gamma_f)*s**3/(gamma_f*s**2 + lambda*s - s**3)

In [22]:
adiabatically_eliminate(tf.subs(lmbda, 0).simplify(), gamma_1).simplify()

sqrt(2)*(gamma_f + s)/(2*sqrt(gamma_f))

In [23]:
a1, a1d, ain_1 = eqns.get_symbols(['a_1', 'conjugate(a_1)', 'ain_1'])
ain1 = eqns.solve([a1, a1d])[ain_1]
tf = (a1 / ain1).simplify()
gamma_1, _ = split_network.aux_coupling_constants
tf = adiabatically_eliminate(tf, gamma_1)
tf

sqrt(2)*sqrt(gamma_f)*s/(gamma_f*s + lambda - s**2)

In [24]:
1 / (tf.subs(lmbda, 0).simplify())

sqrt(2)*(gamma_f - s)/(2*sqrt(gamma_f))

In [25]:
a2, a2d, aout_1 = eqns.get_symbols(['a_2', 'conjugate(a_2)', 'aout_1'])
aout1 = eqns.solve([a2, a2d])[aout_1]
tf = (aout1 / a2d).simplify()
gamma_1, _ = split_network.aux_coupling_constants
tf = adiabatically_eliminate(tf, gamma_1)
tf

(-sqrt(2)*gamma_f*s + sqrt(2)*lambda - sqrt(2)*s**2)/(2*sqrt(gamma_f)*sqrt(lambda))